# Convolution Neural Net

In [2]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.examples.tutorials.mnist import input_data
FLAGS = None
mnist = input_data.read_data_sets('./', one_hot=True)

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


In [55]:
#Clearing all
tf.reset_default_graph()
#Opening new directory for tensorboard
from datetime import datetime
now = datetime.now()
logdir = 'C:/Users/CK/Documents/tf_logs' + now.strftime("%Y%m%d-%H%M%S") + "/"
#To monitor training through tensorboard
print('TensorBoard CMD:tensorboard --logdir='+logdir)

TensorBoard CMD:tensorboard --logdir=C:/Users/CK/Documents/tf_logs20181011-151422/


In [5]:
learning_rate = 0.01
epochs = 20
batch_size = 800
inputs_ = tf.placeholder(tf.float32, (batch_size, 28, 28, 1), name='inputs')

## Example 1

In [6]:
def featConstructor(inputs,varScope,batch_normalised=True,activation=tf.nn.sigmoid):
    
    with tf.variable_scope(varScope):
        norm1=tf.layers.batch_normalization(inputs_,training=batch_normalised,axis=-1)
        conv1 = tf.layers.separable_conv2d(
                norm1,
                filters=32,
                kernel_size=(3,3),
                strides=(1, 1),
                activation=activation
        )
        # Now 28x28x32
        maxpool1 = tf.layers.max_pooling2d(conv1, pool_size=(2,2), strides=(2,2), padding='same')
        # Now 14x14x32
        norm2=tf.layers.batch_normalization(maxpool1,training=batch_normalised,axis=-1)
        conv2 = tf.layers.separable_conv2d(
                inputs=norm2,
                filters=16,
                kernel_size=(3,3),
                strides=(1, 1),
                activation=activation
        )
        # Now 14x14x16
        maxpool2 = tf.layers.max_pooling2d(conv2, pool_size=(2,2), strides=(2,2), padding='same')
        # Now 7x7x16
        norm3=tf.layers.batch_normalization(maxpool2,training=batch_normalised,axis=-1)
        conv3= tf.layers.separable_conv2d(
                inputs=norm3,
                filters=8,
                kernel_size=(3,3),
                strides=(1, 1),
                activation=activation
        )
        # Now 7x7x8
        return tf.layers.max_pooling2d(conv3, pool_size=(2,2), strides=(2,2), padding='same',name='encoded') # Now 4x4x8


#### Image Constructor

In [7]:

def imgConstructor(features,varScope,batch_normalised=True, activation=tf.nn.relu):
    with tf.variable_scope(varScope):
        # Originally 4x4x8
        norm1=tf.layers.batch_normalization(
            features,
            training=batch_normalised,
            axis=-1
        )
        upsample1 = tf.image.resize_images(norm1, size=(7,7), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
        # Now 7x7x8
        conv4 = tf.layers.separable_conv2d(inputs=upsample1,
                filters=16,
                kernel_size=(3,3),
                strides=(1, 1),
                padding='same',
                data_format='channels_last',
                dilation_rate=(1, 1),
                depth_multiplier=2,
                activation=activation)
        norm2=tf.layers.batch_normalization(
            conv4,
            training=batch_normalised,
            axis=-1
        )
        # Now 7x7x16
        upsample2 = tf.image.resize_images(norm2, size=(14,14), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
        # Now 14x14x32
        conv5 = tf.layers.separable_conv2d(inputs=upsample2,
                filters=32,
                kernel_size=(3,3),
                strides=(1, 1),
                padding='same',
                data_format='channels_last',
                dilation_rate=(1, 1),
                depth_multiplier=2,
                activation=activation)
        norm3=tf.layers.batch_normalization(
            conv5,
            training=batch_normalised,
            axis=-1
        )
        # Now 14x14x32
        upsample3 = tf.image.resize_images(norm3, size=(28,28), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
        # Now 28x28x64
        conv6 = tf.layers.separable_conv2d(inputs=upsample3, 
                filters=64,
                kernel_size=(3,3),
                strides=(1, 1),
                padding='same',
                data_format='channels_last',
                dilation_rate=(1, 1),
                depth_multiplier=2,
                activation=activation)
        # Now 28x28x64
        norm4=tf.layers.batch_normalization(
            conv6,
            training=batch_normalised,
            axis=-1
        )
        logits = tf.layers.separable_conv2d(inputs=norm4, filters=1, kernel_size=(3,3), padding='same', activation=None)
        #Now 28x28x1
        # Pass logits through sigmoid to get reconstructed image
        return logits,tf.nn.sigmoid(logits)

In [8]:
def encoder(inputs,featConstructor,imgConstructor,varScope):
    with tf.variable_scope(varScope):
        features=featConstructor(inputs,'features')
        images_logits,images=imgConstructor(features,'images')
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=inputs, logits=images_logits))
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            adam = tf.train.AdamOptimizer(learning_rate)
            tf.summary.scalar('loss',loss)
            tf.summary.image('Input_Images',inputs)
            tf.summary.image('decoded_Images',images)
            # Return the encoded code for embedding visualisation, the optimiser for executing training session.
            return features,adam.minimize(loss,var_list= tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope=varScope))

In [ ]:
#Defining saver
feats,opt=encoder(inputs_,featConstructor,imgConstructor,'CNN_encoder')
saver = tf.train.Saver()
sess = tf.Session()
modelname="CNN_encodernew"

'''
if(os.path.isfile(modelname+".ckpt.index") ):
    saver.restore(sess, "./"+modelname+".ckpt")
else:
    print('Error when loading model, applying initialisation instead.')
    sess.run(tf.global_variables_initializer())
    print('Model initialised.')
'''   
saver.restore(sess, "./"+modelname+".ckpt")
merged=tf.summary.merge_all()
writer = tf.summary.FileWriter(logdir, sess.graph)
for e in range(epochs):
    for ii in range(mnist.train.num_examples//batch_size):
        batch = mnist.train.next_batch(batch_size)
        imgs = batch[0].reshape((-1, 28, 28, 1))
        summary,features, _ = sess.run([merged,feats, opt], feed_dict={inputs_: imgs})
        print("Running epochs {}, batch {}".format(e+1,ii+1))
        writer.add_summary(summary, ii+batch_size*e)

    # Add ops to save and restore all the variables.
    saver.save(sess, "./"+modelname+"new.ckpt")

INFO:tensorflow:Restoring parameters from ./CNN_encodernew.ckpt
Running epochs 1, batch 1
Running epochs 1, batch 2
Running epochs 1, batch 3


In [ ]:
print('TensorBoard CMD:tensorboard --logdir='+logdir)